# Import Packages

In [44]:
import pandas as pd
import numpy as np
import datetime
import random
import time 

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

import xgboost as xgb

from statsmodels.tsa.holtwinters import ExponentialSmoothing

from itertools import chain, combinations
import statsmodels.api as sm

# Data Cleaning

In [18]:
data = pd.read_excel('201908 UNC vs USC(1).xlsx', sheet_name='hist', parse_dates=['Date'])
data['DateTime'] = pd.to_datetime(data['Date']) + pd.TimedeltaIndex(data['Hour'], unit='h')
data.pop('Predicted Load')

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
43843   NaN
43844   NaN
43845   NaN
43846   NaN
43847   NaN
Name: Predicted Load, Length: 43848, dtype: float64

# Create Features

In [19]:
def yt(lag, data_yt):
    this_dataframe = data_yt.shift(periods = lag)
    # Trend
    this_dataframe['Trend'] = range(this_dataframe.shape[0])
    # Month of the year
    this_dataframe['Month'] = this_dataframe['DateTime'].dt.month
    # Day of the week
    this_dataframe['Week'] = this_dataframe['DateTime'].dt.dayofweek
    # Hour of the day
    this_dataframe['Hour'] = this_dataframe['DateTime'].dt.hour
    # I(Week, Hour)
    this_dataframe['Week_Hour'] = this_dataframe['Week'] * this_dataframe['Hour']
    this_dataframe.drop(['DateTime', 'Temperature'], inplace = True, axis = 1)
    return this_dataframe

def fofT(column, lag, data_fofT):
    this_dataframe = data_fofT.shift(periods = lag)
    # Temperatrue
    this_dataframe['Lag_'+ str(lag) +'_' + str(column)] = this_dataframe[column]
    # Temperatrue ** 2
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_2'] = this_dataframe[column] ** 2
    # Temperatrue ** 3
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_3'] = this_dataframe[column] ** 3
    # I(T, Month)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_Month'] = this_dataframe[column] * this_dataframe['Month']
    # I(T_2, Month)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_2_Month'] = this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_2'] * this_dataframe['Month']
    # I(T_3, Month)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_3_Month'] = this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_3'] * this_dataframe['Month']
    # I(T, Hour)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_Hour'] = this_dataframe[column] * this_dataframe['Hour']
    # I(T_2, Hour)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_2_Hour'] = this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_2'] * this_dataframe['Hour']
    # I(T_3, Hour)
    this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_3_Hour'] = this_dataframe['Lag_'+ str(lag) +'_' + str(column) + '_3'] * this_dataframe['Hour']
    try:
        this_dataframe.drop(['DateTime', 'Month', 'Hour', column], inplace = True, axis = 1)
    except:
        pass
    return this_dataframe

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [20]:
# build the dataframe for fofT
data_fofT = data.copy()
data_fofT.drop(['Date', 'Hour', 'Load'], inplace = True, axis = 1)
data_fofT['Month'] = data_fofT['DateTime'].dt.month
data_fofT['Hour'] = data_fofT['DateTime'].dt.hour
# data_fofT.set_index('DateTime', inplace = True)
data_yt = data[['Load', 'Temperature', 'DateTime']][:]
data_rolling = data_fofT.copy()
data_rolling['Temperature_Rolling_last_24hour'] = data_rolling['Temperature'].shift(1).rolling(window=24).mean()
data_rolling.drop('Temperature', inplace = True, axis = 1)
base_data = yt(0, data_yt)
X_build_model = base_data[:]
for h in range(73):
    lag_h = fofT('Temperature', h, data_fofT)
    X_build_model = pd.concat([X_build_model, lag_h], axis = 1)
for day in range(1, 8):
    lag_day = fofT('Temperature_Rolling_last_24hour', (day-1) * 24, data_rolling)
    X_build_model = pd.concat([X_build_model, lag_day], axis = 1)

In [21]:
X_build_model.index = data['DateTime']

In [22]:
X_train = X_build_model[:'2010-12-31'].dropna()
X_train_x, X_train_y = X_train.drop('Load', axis = 1), X_train['Load']
X_test = X_build_model['2011-01-01':'2011-12-31'].dropna()
X_test_x, X_test_y = X_test.drop('Load', axis = 1), X_test['Load']
estimator = LinearRegression()
selector = RFE(estimator, 50, step=1)
selector = selector.fit(X_train_x, X_train_y)
X_train_x_Sed = X_train_x[X_train_x.columns[selector.support_]]
X_test_x_Sed = X_test_x[X_test_x.columns[selector.support_]]

# Build Model

In [42]:
depth = list(range(1, 7))
estimators = list(range(100, 501, 50))
learning = list(map(lambda a: a/10.0, range(1, 5)))
res = dict()

In [45]:
for d in depth:
    for e in estimators:
        for l in learning:
            start = time.time()
            print((d, e, l))
            res[(d, e, l)] = dict()
            model = xgb.XGBRegressor(max_depth=d, 
                             learning_rate=l, 
                             n_estimators=e, 
                             silent=True, 
                             objective="reg:linear", 
                             eval_metric='rmse', 
                             seed=0, 
                             reg_alpha=0.1)
            model.fit(X_train_x_Sed, X_train_y)
            res[(d, e, l)]['Training MAPE'] = mape(X_train_y, model.predict(X_train_x_Sed)) 
            res[(d, e, l)]['Testing MAPE'] = mape(X_test_y, model.predict(X_test_x_Sed)) 
            res[(d, e, l)]['Diff'] = mape(X_test_y, model.predict(X_test_x_Sed)) - mape(X_train_y, model.predict(X_train_x_Sed)) 
            res[(d, e, l)]['Score'] = 0.5 * mape(X_test_y, model.predict(X_test_x_Sed)) + 0.5 * mape(X_train_y, model.predict(X_train_x_Sed))
            end = time.time()
            res[(d, e, l)]['Used'] = end - start
            print(f'used: {end - start}')
            print('---------------')

(1, 100, 0.1)
used: 3.2105278968811035
---------------
(1, 100, 0.2)
used: 3.0785348415374756
---------------
(1, 100, 0.3)
used: 3.039285898208618
---------------
(1, 100, 0.4)
used: 3.0818607807159424
---------------
(1, 150, 0.1)
used: 4.513020992279053
---------------
(1, 150, 0.2)
used: 4.486410617828369
---------------
(1, 150, 0.3)
used: 4.464382171630859
---------------
(1, 150, 0.4)
used: 4.455207109451294
---------------
(1, 200, 0.1)
used: 5.882624864578247
---------------
(1, 200, 0.2)
used: 5.917168855667114
---------------
(1, 200, 0.3)
used: 5.916374683380127
---------------
(1, 200, 0.4)
used: 5.996528148651123
---------------
(1, 250, 0.1)
used: 7.406002998352051
---------------
(1, 250, 0.2)
used: 7.362830638885498
---------------
(1, 250, 0.3)
used: 7.2975380420684814
---------------
(1, 250, 0.4)
used: 7.4226157665252686
---------------
(1, 300, 0.1)
used: 8.668368101119995
---------------
(1, 300, 0.2)
used: 8.936366081237793
---------------
(1, 300, 0.3)
used: 8.7

used: 10.873230934143066
---------------
(5, 200, 0.1)
used: 15.261389970779419
---------------
(5, 200, 0.2)
used: 14.751626253128052
---------------
(5, 200, 0.3)
used: 14.448282718658447
---------------
(5, 200, 0.4)
used: 14.811059951782227
---------------
(5, 250, 0.1)
used: 18.84925389289856
---------------
(5, 250, 0.2)
used: 18.205584049224854
---------------
(5, 250, 0.3)
used: 18.44178295135498
---------------
(5, 250, 0.4)
used: 18.56339693069458
---------------
(5, 300, 0.1)
used: 23.421704053878784
---------------
(5, 300, 0.2)
used: 23.385234117507935
---------------
(5, 300, 0.3)
used: 22.93981695175171
---------------
(5, 300, 0.4)
used: 23.862837076187134
---------------
(5, 350, 0.1)
used: 27.786980867385864
---------------
(5, 350, 0.2)
used: 27.253559112548828
---------------
(5, 350, 0.3)
used: 27.0350821018219
---------------
(5, 350, 0.4)
used: 27.361713886260986
---------------
(5, 400, 0.1)
used: 31.289059162139893
---------------
(5, 400, 0.2)
used: 31.3519670

In [46]:
results = pd.DataFrame(res)

In [59]:
results.transpose().sort_values('Diff').head(20)

Training MAPE  Testing MAPE      Diff     Score       Used
1 100 0.1       6.289957      6.232574 -0.057383  6.261266   3.210528
  150 0.1       5.498315      5.458428 -0.039887  5.478371   4.513021
  200 0.1       5.052667      5.033821 -0.018846  5.043244   5.882625
  250 0.1       4.774059      4.756255 -0.017804  4.765157   7.406003
  100 0.2       5.048665      5.036987 -0.011678  5.042826   3.078535
  300 0.1       4.589513      4.580709 -0.008804  4.585111   8.668368
  200 0.2       4.413354      4.408798 -0.004556  4.411076   5.917169
  150 0.2       4.617687      4.613880 -0.003807  4.615783   4.486411
  350 0.1       4.456983      4.453340 -0.003643  4.455162  10.176587
  400 0.1       4.373313      4.376108  0.002794  4.374710  11.648669
  100 0.3       4.733742      4.738889  0.005146  4.736316   3.039286
  250 0.2       4.285362      4.294070  0.008708  4.289716   7.362831
  450 0.1       4.303080      4.312348  0.009268  4.307714  12.918594
  500 0.1       4.244365      4.257117  0.012752  4.250741  14.385747
  150 0.3       4.463250      4.480251  0.017001  4.471750   4.464382
2 100 0.1       3.787726      3.807064  0.019337  3.797395   4.182350
1 300 0.2       4.191041      4.213824  0.022784  4.202433   8.936366
  350 0.2       4.110783      4.143214  0.032431  4.126999  10.196120
  200 0.3       4.294962      4.333369  0.038407  4.314166   5.916375
  250 0.3       4.173601      4.218043  0.044442  4.195822   7.297538

# Results

In [57]:
temp = X_build_model.dropna()
y = temp['Load']
X = temp.drop('Load', axis = 1).loc[:, selector.support_]
model = xgb.XGBRegressor(max_depth=6, 
                         learning_rate=0.2, 
                         n_estimators=500, 
                         silent=True, 
                         objective="reg:linear", 
                         eval_metric='rmse', 
                         seed=0, 
                         reg_alpha=0.1)

model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
             gamma=0, importance_type='gain', learning_rate=0.2,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
             n_estimators=500, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1,
             seed=0, silent=True, subsample=1, verbosity=1)

In [58]:
mape(y, model.predict(X)) 

0.6279166688598427

In [60]:
X_all = X_build_model.drop('Load', axis = 1)
X_all = X_all.loc[:, selector.support_]
missing = max(X_all.isna().sum())
X_all.dropna(inplace = True)
predictions = model.predict(X_all)
predictions = [np.nan] * missing + predictions.tolist()
data['Prediction'] = predictions

In [61]:
data.tail()

,Date,Hour,Temperature,Load,DateTime,Prediction
43843,2012-12-31,20,22.33,NaN,2012-12-31 20:00:00,14353.001953
43844,2012-12-31,21,20.67,NaN,2012-12-31 21:00:00,14219.715820
43845,2012-12-31,22,19.67,NaN,2012-12-31 22:00:00,13832.237305
43846,2012-12-31,23,19.33,NaN,2012-12-31 23:00:00,13330.486328
43847,2012-12-31,24,18.33,NaN,2013-01-01 00:00:00,11864.851562


In [64]:
results.transpose().to_csv('Results.csv')

In [63]:
data.to_csv('Prediction.csv')